In [ ]:
import json


In [ ]:
# load everything from json, at least 2GB of ram is needed for this
game = {}
# all keys inside game params
keys = {}
with open('tools/GameParams.json', 'r') as j:
    game = json.load(j)
    keys = game.keys()
    # just to make sure something is printed and everything are read
    print(len(keys))

ships = {}
with open('tools/ship_all.json', 'r') as j:
    ships = json.load(j)
    print(len(ships))


In [ ]:
# write json object to disk
def writeToJson(json_dict, name):
    with open(name, 'w+') as l:
        l.write(json.dumps(json_dict) + '\n')

In [ ]:
# find keys in object, can be improved to go deep
def findKey(name, keys):
    temp = []
    for key in keys:
        if name.lower() in key.lower():
            temp.append(key)
    return temp

In [ ]:
# find the first object from the dict
def findFirstObject(name, dic, rootMode=False):
    if isinstance(dic, dict):
        for key in dic.keys():
            temp = dic[key]
            # check if this is the object we want
            if name == key:
                if rootMode is True:
                    return dic
                else:
                    return temp
            result = findFirstObject(name, temp, rootMode)
            if result != None:
                return result
    elif isinstance(dic, list):
        # go through the list
        for value in dic:
            result = findFirstObject(name, value, rootMode)
            if result != None:
                return result

In [ ]:
# Find unique update and their ship
unique_upgrades = findKey('Special_Mod', keys)
# this adds ship id to legendary upgrade
legendary = {}
# this add legendary upgrades to ship
ship_upgrade = {}
# get their id and the ship they belong to
for upgrade in unique_upgrades:
    temp = game[upgrade]
    upgrade_id = temp['id']
    upgrade_id_str = str(upgrade_id)
    # remember to add slot later
    slot = temp['slot']
    
    legendary[upgrade_id_str] = []
    for ship in temp['ships']:
        ship_id = game[ship]['id']
        ship_id_str = str(ship_id)
        
        # init
        if not ship_id_str in ship:
            ship_upgrade[ship_id_str] = {"legendary_upgrades": []}

        # you need to the slot as well
        ship_upgrade[ship_id_str]["legendary_upgrades"].append({"upgrade_id": upgrade_id_str, "slot": slot})
        legendary[upgrade_id_str].append({"ship_id": ship_id, "slot": slot})

writeToJson(legendary, 'legendary_upgrade.json')
writeToJson(ship_upgrade, 'ship_legendary_upgrade.json')


In [ ]:
# find all keys from data
findKey('PZSD508', keys)

In [ ]:
# save anything to json
writeToJson(game['PZSD508_LoYang'], 'lo_yang.json')

In [ ]:
# preview anything
game['PCY006_SmokeGenerator']

In [ ]:
def retrieveDataFromShip(ship):
    temp = {}
    # extract permo flages
    permoflages = ship['permoflages'].copy()
    nativePermoflage = ship['nativePermoflage']
    permoflages.append(nativePermoflage)
    # we need to get all permoflage_id
    permo_ids = []
    for permo in permoflages:
        if permo != '':
            permo_ids.append(game[permo]['id'])
    # add it to temp, there is at least one
    if len(permo_ids) > 0:
        temp['permoflages'] = permo_ids

    # extract HE penetration
    artillery = findFirstObject('HitLocationArtillery', ship, rootMode=True)
    if artillery != None:
        ammoList = artillery['ammoList']
        he = [x for x in ammoList if '_HE' in x]
        # some ships doesn't have HE
        if len(he) > 0:
            # add it to temp
            temp['alphaPiercingHE'] = game[he[0]]['alphaPiercingHE']

    # get sigma
    sigma = findFirstObject('ammoPool', ship, rootMode=True)
    if sigma != None:
        temp['sigma'] = sigma['sigmaCount']

    # save consumables
    ability = findFirstObject('ShipAbilities', ship)
    if ability != None:
        temp['consumables'] = []
        # append slot and abilities in
        for slot in ability:
            s = ability[slot]
            # x[0] is the name of the consumable
            abils = [x for x in s['abils'] if x[0].endswith('Premium')]
            o = []
            for a in abils:
                o.append({'name': a[0], 'type': a[1]})
            # o is used to make a list of objects
            if len(o) > 0:
                # check there is at least one inside, not all ships have 5 consumables
                temp['consumables'].append(o)

    return temp

In [ ]:
# loop through all ship 
additional_ship_info = {}
for ship in ships:
    ship_str = ships[ship]['ship_id_str']
    # it should only has one
    additional_ship_info[ship] = retrieveDataFromShip(game[findKey(ship_str, keys)[0]])
writeToJson(additional_ship_info, 'ship_additional.json')
